In [1]:
import keras,os
from keras.models import Sequential
from keras.layers import Dense, Conv2D, MaxPool2D , Flatten
from keras.preprocessing.image import ImageDataGenerator
import numpy as np

In [2]:
! pip install kaggle
! mkdir ~/.kaggle
! cp kaggle.json ~/.kaggle/
! chmod 600 ~/.kaggle/kaggle.json
! kaggle competitions download nn22-weather-analysis-using-image-recognition

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
 99% 581M/586M [00:06<00:00, 127MB/s]
100% 586M/586M [00:06<00:00, 101MB/s]


In [3]:
from zipfile import ZipFile
with ZipFile('/content/nn22-weather-analysis-using-image-recognition.zip', 'r') as zipObj:
  # Extract all the contents of zip file in current directory
   zipObj.extractall()

In [4]:
import os
from os import listdir
import numpy as np
sum=0
path='/content/Dataset/Train'
Train_list = sorted(listdir(path))
for i in Train_list:
  l=listdir(os.path.join(path, i))
  sum+=len(l)
  print(len(l)," ",i)

print(sum)

557   dew
750   fogsmog
334   frost
538   glaze
450   hail
276   lightning
385   rain
131   rainbow
1059   rime
551   sandstorm
520   snow
5551


In [1]:
import tensorflow as tf
from tensorflow import keras

In [6]:
# FOR DATA AUGMENTATION
train_datagen = tf.keras.preprocessing.image.ImageDataGenerator(
      rotation_range=30,
      width_shift_range=0.2, # Shifting image width by 20%
      height_shift_range=0.2,# Shifting image height by 20%
      shear_range=0.2,       # Rotation across X-axis by 20%
      # zoom_range=0.3,        # Image zooming by 30%
      horizontal_flip=True,
      fill_mode='reflect')

In [8]:
# AUGMENT A SPECIFIC FOLDER
def augment(index,label_name,copies,count):
  f = Train_list[index]
  img_path = os.path.join(path, f)
  c=0
  x=0
  for i in listdir(img_path):
    img=os.path.join(img_path, i)
    img_data = keras.preprocessing.image.load_img(img)
    img_data=keras.preprocessing.image.img_to_array(img_data)
    img_data=img_data.reshape((1,)+img_data.shape)
    y=0
    for b in train_datagen.flow(img_data,batch_size=1,save_to_dir=img_path,save_prefix=label_name,save_format='jpg'):
      y+=1
      if y==copies:
        break
    c+=1
    if count!=-1:
      x+=1
      if x==count:
        break
  l2=listdir(os.path.join(path, Train_list[index]))
  print(len(l2)," ",Train_list[index])

In [9]:
augment(7,'rainbow',5,-1)

762   rainbow


In [10]:
augment(5,'lightning',2,-1)

813   lightning


In [11]:
augment(2,'frost',2,-1)

976   frost


In [12]:
augment(0,'dew',1,220)

774   dew


In [13]:
augment(3,'glaze',1,265)

799   glaze


In [14]:
augment(4,'hail',1,355)

797   hail


In [15]:
augment(6,'rain',1,-1)

764   rain


In [16]:
augment(10,'snow',1,285)

799   snow


In [17]:
augment(9,'sandstorm',1,255) 

802   sandstorm


In [18]:
train_ds = tf.keras.utils.image_dataset_from_directory(
  path,
  seed=42,
  image_size=(224, 224),
  validation_split=0.001,
  subset="training",
  batch_size=1)

Found 9095 files belonging to 11 classes.
Using 9086 files for training.


In [19]:
val_ds = tf.keras.utils.image_dataset_from_directory(
  path,
  seed=42,
  image_size=(224, 224),
  validation_split=0.001,
  subset="validation",
  batch_size=1)

Found 9095 files belonging to 11 classes.
Using 9 files for validation.


In [22]:
#class_names = train_ds.class_names
#print(class_names)

In [23]:
# import matplotlib.pyplot as plt

# plt.figure(figsize=(10, 10))
# for images, labels in train_ds.take(1):
#   for i in range(9):
#     ax = plt.subplot(3, 3, i + 1)
#     plt.imshow(images[i].numpy().astype("uint8"))
#     plt.title(labels[i].numpy())
#     plt.axis("off")


In [32]:
def process_images(image, label):
    # Normalize images to have a mean of 0 and standard deviation of 1
    image = tf.image.per_image_standardization(image)
    return image, label

In [2]:
# VGG19 ARCITECHTURE
from keras.models import Sequential
from keras.layers.core import Flatten, Dense, Dropout
from keras.layers.convolutional import Convolution2D, MaxPooling2D, ZeroPadding2D
model = Sequential()
model.add(Convolution2D(64, 3, 3, activation='relu',input_shape=(224,224,3)))
#model.add(Convolution2D(64,(3,3),padding="same", activation="relu"))
model.add(Convolution2D(filters=64,kernel_size=(3,3),padding="same", activation="relu"))
model.add(MaxPooling2D(pool_size=(2,2),strides=(2,2)))
    
model.add(Convolution2D(filters=128,kernel_size=(3,3),padding="same", activation="relu"))
model.add(Convolution2D(filters=128,kernel_size=(3,3),padding="same", activation="relu"))
model.add(MaxPooling2D(pool_size=(2,2),strides=(2,2)))
    
model.add(Convolution2D(filters=256,kernel_size=(3,3),padding="same", activation="relu"))
model.add(Convolution2D(filters=256,kernel_size=(3,3),padding="same", activation="relu"))
model.add(Convolution2D(filters=256,kernel_size=(3,3),padding="same", activation="relu"))
    #added
model.add(Convolution2D(filters=256,kernel_size=(3,3),padding="same", activation="relu"))
model.add(MaxPooling2D(pool_size=(2,2),strides=(2,2)))
    
model.add(Convolution2D(filters=512,kernel_size=(3,3),padding="same", activation="relu"))
model.add(Convolution2D(filters=512,kernel_size=(3,3),padding="same", activation="relu"))
model.add(Convolution2D(filters=512,kernel_size=(3,3),padding="same", activation="relu"))
    #ADDED
model.add(Convolution2D(filters=512,kernel_size=(3,3),padding="same", activation="relu"))
model.add(MaxPooling2D(pool_size=(2,2),strides=(2,2)))
    
model.add(Convolution2D(filters=512,kernel_size=(3,3),padding="same", activation="relu"))
model.add(Convolution2D(filters=512,kernel_size=(3,3),padding="same", activation="relu"))
model.add(Convolution2D(filters=512,kernel_size=(3,3),padding="same", activation="relu"))
      #ADDED
model.add(Convolution2D(filters=512,kernel_size=(3,3),padding="same", activation="relu"))
model.add(MaxPooling2D(pool_size=(2,2),strides=(2,2)))

In [3]:
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d (Conv2D)              (None, 74, 74, 64)        1792      
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 74, 74, 64)        36928     
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 37, 37, 64)        0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 37, 37, 128)       73856     
_________________________________________________________________
conv2d_3 (Conv2D)            (None, 37, 37, 128)       147584    
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 18, 18, 128)       0         
_________________________________________________________________
conv2d_4 (Conv2D)            (None, 18, 18, 256)       2

In [11]:
def process_images2(image):
    # Normalize images 
    image = tf.image.per_image_standardization(image)
    return image

In [36]:
train_ds = (train_ds.map(process_images))               
val_ds = (val_ds.map(process_images))            

In [4]:
model.add(Flatten())
model.add(Dense(units=11, activation="softmax"))
model.compile(
    loss=keras.losses.sparse_categorical_crossentropy,
    optimizer=tf.keras.optimizers.SGD(0.0001),
    metrics=['accuracy']    
)

In [37]:
# LOADING THE PRETRAINED WEIGHTS FOR IMAGENET
WEIGHTS_PATH_NO_TOP = ('https://github.com/fchollet/deep-learning-models/'
                       'releases/download/v0.1/'
                       'vgg19_weights_tf_dim_ordering_tf_kernels_notop.h5')
weights_path = tf.keras.utils.get_file('vgg19_weights_tf_dim_ordering_tf_kernels_notop.h5',WEIGHTS_PATH_NO_TOP)
model.load_weights(weights_path)
model.add(Flatten())
model.add(Dense(units=11, activation="softmax"))
model.compile(
    loss=keras.losses.sparse_categorical_crossentropy,
    optimizer=tf.keras.optimizers.SGD(0.0001),
    metrics=['accuracy']    
)
# #reduce_lr = tf.keras.callbacks.ReduceLROnPlateau(monitor='val_loss', min_lr=0.00001)
callback = tf.keras.callbacks.EarlyStopping(monitor='val_loss',patience=4)
reduce_lr = tf.keras.callbacks.ReduceLROnPlateau(monitor='val_loss', min_lr=0.0000001,patience=2)
history= model.fit(
      train_ds,
      epochs=10,validation_data=val_ds,callbacks=[reduce_lr,callback] )


# model.summary()

80150528/80134624 [==============================] - 1s 0us/step
Epoch 1/10
9086/9086 [==============================] - 209s 22ms/step - loss: 0.8217 - accuracy: 0.7200 - val_loss: 0.5763 - val_accuracy: 0.8889 - lr: 1.0000e-04
Epoch 2/10
9086/9086 [==============================] - 199s 22ms/step - loss: 0.3960 - accuracy: 0.8653 - val_loss: 0.6155 - val_accuracy: 0.8889 - lr: 1.0000e-04
Epoch 3/10
9086/9086 [==============================] - 198s 22ms/step - loss: 0.2507 - accuracy: 0.9187 - val_loss: 0.6978 - val_accuracy: 0.8889 - lr: 1.0000e-04
Epoch 4/10
9086/9086 [==============================] - 197s 22ms/step - loss: 0.1267 - accuracy: 0.9605 - val_loss: 0.7517 - val_accuracy: 0.8889 - lr: 1.0000e-05
Epoch 5/10
9086/9086 [==============================] - 197s 22ms/step - loss: 0.0982 - accuracy: 0.9712 - val_loss: 0.8301 - val_accuracy: 0.8889 - lr: 1.0000e-05


In [6]:
# import cv2
import os
from os import listdir
import numpy as np
testing_data=[]
import pandas as pd
imagename=[]

# path='/content/Dataset/Test'
path="D:\\university\\p\\ProjectTest"
c=0
for f in listdir(path):
  img= os.path.join(path, f)
  
  try:

  #  img_data = cv2.imread(img, 1)
  #  img_data=cv2.cvtColor(img_data,cv2.COLOR_BGR2RGB)
  #  img_data = cv2.resize(img_data, (224, 224))

   img_data = keras.preprocessing.image.load_img(img,target_size=(224,224))
  #  img_data=keras.preprocessing.image.img_to_array(img_data)
   
  except:
    print(img,'error')
    continue

  testing_data.append(np.array(img_data))
  imagename.append(f)
  c+=1
     
     
df = pd.DataFrame (imagename, columns = ['image_name'])
print(c) 


5


In [7]:
df

,image_name
0,test_1301.jpg
1,test_1302.jpg
2,test_1303.jpg
3,test_1304.jpg
4,test_1305.jpg


In [8]:
testing_data=np.array(testing_data)

In [9]:
test_ds = tf.data.Dataset.from_tensor_slices(testing_data)

In [10]:
model.load_weights("C:\\Users\\Sherif Essam\\OneDrive\\Desktop\\weights_0.86.h5")

In [12]:
test_ds = (test_ds.map(process_images2))

In [13]:
test_ds = (test_ds.batch(batch_size=1, drop_remainder=False))

In [14]:
p=model.predict(test_ds)
pp=np.argmax(p,axis=1)
print(pp)

[ 8  0 10  6  6]


In [15]:
df2 = pd.DataFrame (pp, columns = ['label'])
# new_row = pd.Series(data={'image_name':'test_1297.jpg', 'label':'10'},name='1300')
df2

,label
0,8
1,0
2,10
3,6
4,6


In [33]:
df2['label'].unique()

array([10,  6,  1,  7,  2,  9,  8,  5,  0,  4,  3], dtype=int64)

In [16]:
frames = [df, df2]
result = pd.concat([df, df2], axis=1, join='inner')
# result =result.append(new_row)

In [17]:
result

,image_name,label
0,test_1301.jpg,8
1,test_1302.jpg,0
2,test_1303.jpg,10
3,test_1304.jpg,6
4,test_1305.jpg,6


In [53]:
result.to_csv('A1_1(vgg).csv', index=False)

In [54]:
model.save_weights('weights.h5')

In [55]:
# from keras.models import load_model
# model.save('model.h5')
# model = load_model('model.h5')